# goal strukture 

## phase one: 
>track_name, track_artist, id, lyrics (empty)

## phase two: 
>track_name, track_artist, lyrics, id

## phase three: 
>track_name, track_artist, lyrics, embedding, id


# Phase one preprocessing

In [2]:
import pandas as pd 
import numpy as np

# importing the raw data sets 

df_MSSS = pd.read_csv("../raw_data/Most_Streamed_Spotify_Songs_2024.csv", encoding='latin1')
df_SMSD = pd.read_csv("../raw_data/spotify_millsongdata.csv")
df_SS = pd.read_csv("../raw_data/spotify_songs.csv")

# cleaning them to only the two rows we care about 

df_MSSS_reduced = df_MSSS[["Artist", "Track"]].rename(columns={"Artist": "track_artist", "Track": "track_name"})
df_MSSS_reduced["track_lyrics"] = None

df_SMSD_reduced = df_SMSD[["artist", "song", "text"]].rename(columns={"artist": "track_artist", "song": "track_name", "text": "track_lyrics"}) # also has the text so here we are goingt to put the text from the stuff here already
df_SS_reduced = df_SS[["track_name", "track_artist"]]
df_SS_reduced["track_lyrics"] = None


# combine dfs 

df_base = df_MSSS_reduced
df_base = pd.concat([df_base, df_SMSD_reduced], ignore_index=True)
df_base = pd.concat([df_base, df_SS_reduced], ignore_index=True)

# del duplicates 
df_clean = df_base.drop_duplicates(subset=['track_artist', 'track_name'])

# adding ids

import uuid

def generate_uuid():
    return str(uuid.uuid4())

df_clean['track_id'] = [generate_uuid() for _ in range(len(df_clean))]

df = df_clean


# about 86000 rows 

C:\Users\lipka\AppData\Local\Temp\ipykernel_9640\1460722427.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SS_reduced["track_lyrics"] = None
C:\Users\lipka\AppData\Local\Temp\ipykernel_9640\1460722427.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['track_id'] = [generate_uuid() for _ in range(len(df_clean))]


# Phase two preprocessing

In [2]:
from dotenv import load_dotenv
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd

load_dotenv()

df = pd.read_csv("big_song_data.csv")

try: 
    df = df.drop('Unnamed: 0', axis=1)
except:
    pass


API_TOKEN = os.getenv('client_access_token')
BASE_URL = 'https://api.genius.com'


def search_song(artist: str, title: str):
    search_url = f"{BASE_URL}/search"
    headers = {'Authorization': f'Bearer {API_TOKEN}'}
    params = {'q': f"{title} {artist}"}
    response = requests.get(search_url, headers=headers, params=params)
    return response.json()

# Function to get song lyrics
def get_lyrics(song_api_path: str):
    song_url = f"{BASE_URL}{song_api_path}"
    headers = {'Authorization': f'Bearer {API_TOKEN}'}
    response = requests.get(song_url, headers=headers)
    json_response = response.json()
    path = json_response['response']['song']['path']
    
    # Fetch the song lyrics from the Genius website (not directly available via API)
    page_url = f"https://genius.com{path}"
    page = requests.get(page_url)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Find the lyrics container with the data-lyrics-container="true" attribute
    lyrics_div = soup.find('div', {'data-lyrics-container': 'true'})
    lyrics = lyrics_div.get_text(separator='\n') if lyrics_div else "Lyrics not found."
    return lyrics

def get_lyrics_for_song(artist: str, title: str, iteration: int) -> str:
    search_result = search_song(artist, title)
    if search_result['response']['hits']:
        song_api_path = search_result['response']['hits'][0]['result']['api_path']
        print(f"{iteration} - FOUND - Title: {title} - Artist: {artist}")
        return get_lyrics(song_api_path)            
    print(f"{iteration} - FAIL - Title: {title} - Artist: {artist}")
    print(search_result)
    return None

# iterating through the df in batches - something like 500 songs 
# throwing it at threads because its a task that can be perfectly threaded
# saving the batches to save progress 

for idx, row in df.iterrows():
    if not pd.isna(row['track_lyrics']):
        continue

    artist = row["track_artist"]
    track_name = row["track_name"]

    lyrics = get_lyrics_for_song(artist, track_name, idx)
    df.at[idx, 'track_lyrics'] = lyrics
    df.to_csv("big_song_data.csv", index=False)



8 - FAIL - Title: Danza Kuduro - Cover - Artist: MUSIC LAB JPN
24 - FAIL - Title: Beat Automotivo Tan Tan Tan Viral - Artist: WZ Beat
29 - FAIL - Title: Danza Kuduro - Cover - Artist: LOVE BGM JPN
40 - FAIL - Title: Cupid ï¿½ï¿½ï¿½ Twin Ver. (FIFTY FIFTY) ï¿½ï¿½ï¿½ Spe - Artist: sped up 8282
58 - FAIL - Title: Very Cute Melody by Marimba Tone (39813) - Artist: mitsu sound
73 - FAIL - Title: Laxed ï¿½ï¿½ï¿½ Sire - Artist: Jawsh 685
85 - FAIL - Title: Titï¿½ï¿½ Me Pregu - Artist: Bad Bunny
114 - FAIL - Title: Chegou a Hora de Ir para Cama (Playback) - Artist: 3 Palavrinhas
132 - FAIL - Title: PHONK BRASILEIRO FRESCO (Slowed + Reverb) - Artist: DJ MOIGUS
133 - FAIL - Title: Sooseki (From "Pushpa 2 The Rule") [TELUGU] - Artist: Shreya Ghoshal
135 - FAIL - Title: Casca de Bala - Artist: Thullio Milionï¿½ï¿
138 - FAIL - Title: Jedag Jedug Capcut 2024 - Artist: Afrian Af
143 - FAIL - Title: ýýýýýýýýýýýýýýýýýýýýý - Artist: FLI:P
151 - FAIL - Title: Again - Artist: Kurochuu
157 - FAIL - Title: 

KeyboardInterrupt: 

# Threading

In [9]:
# now the multithread approach
import threading

df = pd.read_csv("big_song_data.csv")

# going here and slicing the original df into equal parts
part_count = 10
len_df = df.shape[0]
slice_size = len_df // len_df
print(slice_size) # what is it ? 

df_name = []


def scrape_lyrics_df(df_path: str):
    print("thread started")
    part_df = pd.read_csv(df_path)

    for idx, row in part_df.iterrows():
        if not pd.isna(row['track_lyrics']):
            continue

        artist = row["track_artist"]
        track_name = row["track_name"]

        lyrics = get_lyrics_for_song(artist, track_name, idx)
        part_df.at[idx, 'track_lyrics'] = lyrics
        part_df.to_csv(df_path, index=False)

for i in range(part_count):
    if i != part_count-1:
        print("cut here")
        df_part = df.iloc[i*slice_size:(i*slice_size-1)]
    else: 
        print("took the rest")
        df_part = df.iloc[i*slice_size:]
    part_name = f"tmp/big_song_data_part_{i}.csv"
    df_part.to_csv(part_name, index=False)
    df_name.append(part_name)

for df_path in df_name:
    print(df_path)
    a = threading.Thread(target=scrape_lyrics_df, args=(df_path,))
    a.start()




# give everyt thread one part of the fun 
# do the lyrics search 

# recombine the dfs 
# and save



tmp/big_song_data_part_0.csv
thread started
tmp/big_song_data_part_1.csv
thread started
tmp/big_song_data_part_2.csv
thread started
tmp/big_song_data_part_3.csv
thread started
tmp/big_song_data_part_4.csv
thread started
tmp/big_song_data_part_5.csv
thread started
tmp/big_song_data_part_6.csv
thread started
tmp/big_song_data_part_7.csv
thread started
tmp/big_song_data_part_8.csv
thread started
tmp/big_song_data_part_9.csv
thread started
15 - FAIL - Title: Beat Automotivo Tan Tan Tan Viral - Artist: WZ Beat
8 - FAIL - Title: Danza Kuduro - Cover - Artist: MUSIC LAB JPN


24 - FAIL - Title: Beat Automotivo Tan Tan Tan Viral - Artist: WZ Beat
20 - FAIL - Title: Danza Kuduro - Cover - Artist: LOVE BGM JPN
31 - FAIL - Title: Cupid ï¿½ï¿½ï¿½ Twin Ver. (FIFTY FIFTY) ï¿½ï¿½ï¿½ Spe - Artist: sped up 8282
29 - FAIL - Title: Danza Kuduro - Cover - Artist: LOVE BGM JPN
49 - FAIL - Title: Very Cute Melody by Marimba Tone (39813) - Artist: mitsu sound
40 - FAIL - Title: Cupid ï¿½ï¿½ï¿½ Twin Ver. (FIFTY FIFTY) ï¿½ï¿½ï¿½ Spe - Artist: sped up 8282
58 - FAIL - Title: Very Cute Melody by Marimba Tone (39813) - Artist: mitsu sound
64 - FAIL - Title: Laxed ï¿½ï¿½ï¿½ Sire - Artist: Jawsh 685
73 - FAIL - Title: Laxed ï¿½ï¿½ï¿½ Sire - Artist: Jawsh 685
76 - FAIL - Title: Titï¿½ï¿½ Me Pregu - Artist: Bad Bunny
85 - FAIL - Title: Titï¿½ï¿½ Me Pregu - Artist: Bad Bunny
105 - FAIL - Title: Chegou a Hora de Ir para Cama (Playback) - Artist: 3 Palavrinhas
114 - FAIL - Title: Chegou a Hora de Ir para Cama (Playback) - Artist: 3 Palavrinhas
123 - FAIL - Title: PHONK BRASILEIRO FRES

Exception in thread Thread-225 (scrape_lyrics_df):
Traceback (most recent call last):
  File "C:\Users\lipka\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\lipka\HARD_DRIVE\Projekte\Projekte\Project 146\current\shuffle.ai\myenv\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\lipka\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\lipka\AppData\Local\Temp\ipykernel_9520\4077521583.py", line 27, in scrape_lyrics_df
  File "c:\Users\lipka\HARD_DRIVE\Projekte\Projekte\Project 146\current\shuffle.ai\myenv\lib\site-packages\pandas\util\_decorators.py", line 333, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\lipka\HARD_DRIVE\Projekte\Projekte\Project 146\current\shuffle.ai\myenv\lib\site-packages\pandas\core\generic.py", line 3967, in to_csv
    retu

539 - FAIL - Title: Broccoli - Artist: ati2x06
540 - FOUND - Title: Angel Baby - Artist: Troye Sivan
541 - FOUND - Title: Way 2 Sexy (with Future & Young Thug) - Artist: Drake
542 - FAIL - Title: Bhauji Lenge Lenge - Artist: Khesari Lal Yadav
543 - FOUND - Title: SORRY 4 THAT MUCH - Artist: Feid
544 - FAIL - Title: I WANNA BE YOUR SLAVE - Artist: Mï¿½ï¿½ne
545 - FOUND - Title: PUFFIN ON ZOOTIEZ - Artist: Future
546 - FOUND - Title: 3D (feat. Jack Harlow) - Artist: Jung Kook
547 - FOUND - Title: XO Tour Llif3 - Artist: Lil Uzi Vert
548 - FOUND - Title: Lift Me Up - From Black Panther: Wakanda Forever - Music From and Inspired By - Artist: Rihanna
549 - FOUND - Title: Desire (with Sam Smith) - Artist: Calvin Harris
550 - FAIL - Title: Cute - Artist: Aurel Surya Lie
551 - FOUND - Title: Cold Heart - PNAU Remix - Artist: Elton John
552 - FOUND - Title: Con Altura - Artist: ROSALï¿½
553 - FAIL - Title: ýýýýýý (HEYA) - Artist: IVE
554 - FOUND - Title: LA CANCIï¿½ - Artist: J Balvin
555 - FOU

Exception in thread Thread-234 (scrape_lyrics_df):
Traceback (most recent call last):
  File "C:\Users\lipka\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\lipka\HARD_DRIVE\Projekte\Projekte\Project 146\current\shuffle.ai\myenv\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\lipka\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\lipka\AppData\Local\Temp\ipykernel_9520\4077521583.py", line 27, in scrape_lyrics_df
  File "c:\Users\lipka\HARD_DRIVE\Projekte\Projekte\Project 146\current\shuffle.ai\myenv\lib\site-packages\pandas\util\_decorators.py", line 333, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\lipka\HARD_DRIVE\Projekte\Projekte\Project 146\current\shuffle.ai\myenv\lib\site-packages\pandas\core\generic.py", line 3967, in to_csv
    retu

595 - FAIL - Title: ýýchame La Culpa - Artist: Luis Fonsi


In [11]:
# combining the dfs again
# something is still messed up with the slicing or something idk but there are to many entrys now 
# somehow doubled the amount of line idk


df_base_combine = pd.read_csv(df_name[0])

for df_path in df_name[1:]:
    df_base_combine = pd.concat([df_base_combine, pd.read_csv(df_path)], ignore_index=True)


df_base_combine.to_csv("big_song_data_test.csv", index=False)

In [15]:
check_df = pd.read_csv("big_song_data_test.csv")

(173428, 4)